<div class="alert alert-block alert-warning">The end result of this exercise should be a file named acquire.py.</div>

In [52]:
import requests
import pandas as pd
import os

### Explore Site and JSON API
- Investigate the documentation provided by the API.
- Explore a couple of responses before I dig into the exercises. 

In [2]:
# Make a request o the url and use `.jason` method to return JSON

base_url = 'https://python.zach.lol'

# Print the dtype of data we retrieve

type(requests.get(base_url).json())

dict

In [3]:
# Print the JSON file

requests.get(base_url).json() # two paths I can add to my base url

{'api': '/api/v1', 'help': '/documentation'}

In [4]:
# Create a doc_url to request help

doc_url = base_url + '/documentation'

In [6]:
# Explore the keys in the dictionary returned from doc_url

requests.get(doc_url).json().keys()

dict_keys(['payload', 'status'])

In [7]:
# Print out the value for the status key

requests.get(doc_url).json()['status']

'ok'

In [9]:
# Print the value for the payload key

print(requests.get(doc_url).json()['payload']) # print() returns a neat output


The API accepts GET requests for all endpoints, where endpoints are prefixed
with

    /api/{version}

Where version is "v1"

Valid endpoints:

- /stores[/{store_id}]
- /items[/{item_id}]
- /sales[/{sale_id}]

All endpoints accept a `page` parameter that can be used to navigate through
the results.



**Takeaways**
1. I can use 3 different endpoints to access data by adding stores, items, sales to my `base_url + /api/v1`
2. A page parameter for navigation through the results
    - `?page=n`

In [16]:
# Create a api_url to request api

api_url = base_url + '/api/v1/'

### 1. Using the code from the lesson as a guide, create a dataframe named items that has all of the data for items.

In [17]:
# Submit the request for the first page of results and store the results in response
response = requests.get(api_url + 'items') # similar to create an object like lm, kmeans, ...

# Check my request
response.ok # succesful

True

In [23]:
# Use .json() method on my response to get a dictionary

items = response.json()

# Print data type and size of the data

print(type(items))
items

<class 'dict'>


{'payload': {'items': [{'item_brand': 'Riceland',
    'item_id': 1,
    'item_name': 'Riceland American Jazmine Rice',
    'item_price': 0.84,
    'item_upc12': '35200264013',
    'item_upc14': '35200264013'},
   {'item_brand': 'Caress',
    'item_id': 2,
    'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
    'item_price': 6.44,
    'item_upc12': '11111065925',
    'item_upc14': '11111065925'},
   {'item_brand': 'Earths Best',
    'item_id': 3,
    'item_name': 'Earths Best Organic Fruit Yogurt Smoothie Mixed Berry',
    'item_price': 2.43,
    'item_upc12': '23923330139',
    'item_upc14': '23923330139'},
   {'item_brand': 'Boars Head',
    'item_id': 4,
    'item_name': 'Boars Head Sliced White American Cheese - 120 Ct',
    'item_price': 3.14,
    'item_upc12': '208528800007',
    'item_upc14': '208528800007'},
   {'item_brand': 'Back To Nature',
    'item_id': 5,
    'item_name': 'Back To Nature Gluten Free White Cheddar Rice Thin Crackers',
    'item_price':

In [24]:
# List the keys in the dictionary

items.keys() # two keys: payload and status

dict_keys(['payload', 'status'])

**Takeways**
1. `payload` is a dictionary.
2. First key `items` is a list of dictionaries. 
3. Check all of the key: value paries

In [26]:
# Check the keys in the payload dictionary
items['payload'].keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [27]:
len(items['payload']['items']) # hold 20 dictionaries

20

In [28]:
# How many pages in the data?
items['payload']['max_page'] # total number of pages is 3

3

In [29]:
# What is the value of 'next_page'
items['payload']['next_page'] # next page

'/api/v1/items?page=2'

In [30]:
# What is the value of 'page'?
items['payload']['page'] # Current page is 1. 

1

In [31]:
# Create a for loop to combine all the pages

# Create a base_url
base_url = 'https://python.zach.lol'

# Load the first page of the data
response = requests.get(base_url + '/api/v1/items')
items = response.json()

# Conver the data to df
df_items = pd.DataFrame(items['payload']['items'])

# For loop the rest of the pages

pages = 2
for page in range(0, pages):
    response = requests.get(base_url + items['payload']['next_page'])
    items = response.json()
    df_items = pd.concat([df_items, pd.DataFrame(items['payload']['items'])])

df_items.shape

(50, 6)

In [32]:
# No reset_index()

df_items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [12]:
# Reset_index()

df.head()

,level_0,index,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,0,0.0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,1.0,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,2,2.0,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,3,3.0,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,4,4.0,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


### 2. Do the same thing, but for stores. 

In [18]:
# What is the key structure for stores?

response = requests.get('https://python.zach.lol/api/v1/stores')

stores = response.json()
stores.keys()

dict_keys(['payload', 'status'])

In [19]:
# Any nested key in 'payload'?

stores['payload'].keys()

dict_keys(['max_page', 'next_page', 'page', 'previous_page', 'stores'])

In [23]:
# What is the max page in stores/payload/payload?
# max page is 1

stores['payload']['max_page']

1

In [25]:
# Conver the data to df

df_stores = pd.DataFrame(stores['payload']['stores'])
df_stores.shape

(10, 5)

### 3. Extract the data for sales. There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [29]:
# What is the structure of the keys for sales?

response = requests.get('https://python.zach.lol/api/v1/sales')

sales = response.json()
sales.keys()

dict_keys(['payload', 'status'])

In [30]:
# Any nested key in 'payload'?

sales['payload'].keys()

dict_keys(['max_page', 'next_page', 'page', 'previous_page', 'sales'])

In [31]:
# What is the max page in stores/payload/payload?
# max page is 183

sales['payload']['max_page']

183

In [34]:
# Create a for loop to combine all the pages

# Create a base_url
base_url = 'https://python.zach.lol'

# Load the first page of the data
response = requests.get(base_url + '/api/v1/sales')
sales = response.json()

# Conver the data to df
df_sales = pd.DataFrame(sales['payload']['sales'])

# For loop the rest of the pages

pages = 182
for page in range(0, pages):
    response = requests.get(base_url + sales['payload']['next_page'])
    sales = response.json()
    df_sales = pd.concat([df_sales, pd.DataFrame(sales['payload']['sales'])])

df_sales.shape

(913000, 5)

### 4. Save the data in your files to local csv files so that it will be faster to access in the future.

In [45]:
# df_items.to_csv('df_items.csv')
# df_stores.to_csv('df_stores.csv')
# df_sales.to_csv('df_sales.csv')

### Build Helper Fucntion

In [34]:
def get_pages(name):
    """
    Return a df containing all pages of items, stores, or sales
    Parameter: string name: 'items', 'stores', or 'sales'
    """
    base_url = 'https://python.zach.lol'
    api_url = base_url + '/api/v1/'
    response = requests.get(api_url + name)
    data = response.json()
    
    # Load the first page
    list_of_pages = data['payload'][name] 
    
    # While Loop the pages and concat them together
    while data['payload']['next_page'] != None:
        response = requests.get(base_url + data['payload']['next_page'])
        data = response.json()
        list_of_pages.extend(data['payload'][name]) # iterates over its argument and adding each element
        
    # Convert the pages to dataframe
    df = pd.DataFrame(list_of_pages)
    return df

In [36]:
# Test the function
df = get_pages('items')

# Print the shape of df
print(df.shape)
df.head()

(50, 6)


,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


### 5. Combine the data from your three separate dataframes into one large dataframe.
- `df1.set_index().join(df2.set_index())`
- `pd.merge(df1, df2, left_on='', right_on='').drop(columns='')`

In [47]:
# Read from .csv
# Take a look at the df_items
# The PK is item_id

df_items.head(1)

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013


In [48]:
# The PK is store_id

df_stores.head(1)

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253


In [77]:
# The PK is sale_id

df_sales.head(1)

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1


In [78]:
# Check the shape of df_items, df_stores_, df_sales
df_items.shape, df_stores.shape, df_sales.shape

((50, 6), (10, 5), (913000, 5))

####  `df.set_index('key').join(other.set_index('key'))`
If we want to join using the key columns, we need to set key to be the index in both df and other. The joined DataFrame will have key as its index.

In [95]:
# df_sales join df_stores on store_id
df = df_sales.set_index('store').join(df_stores.set_index('store_id'))

# then join the df_items on item_id
df = df.set_index('item').join(df_items.set_index('item_id'))

# Randomly check df
df.sample(5) # No store_id and item_id

,sale_amount,sale_date,sale_id,store_address,store_city,store_state,store_zipcode,item_brand,item_name,item_price,item_upc12,item_upc14
33,73.0,"Mon, 25 May 2015 00:00:00 GMT",590673,516 S Flores St,San Antonio,TX,78204,The First Years,Tomy The First Years Gumdrop Orthodontic Pacif...,6.52,71463060078,71463060078
40,27.0,"Mon, 22 May 2017 00:00:00 GMT",713743,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,Amys,Amys Light & Lean Spaghetti Italiano,0.60,42272008063,42272008063
4,33.0,"Sun, 17 Sep 2017 00:00:00 GMT",61979,516 S Flores St,San Antonio,TX,78204,Boars Head,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
8,72.0,"Thu, 24 Oct 2013 00:00:00 GMT",144551,8503 NW Military Hwy,San Antonio,TX,78231,Lea & Perrins,Lea & Perrins Marinade In-a-bag Cracked Pepper...,1.68,51600080015,51600080015
44,26.0,"Wed, 26 Oct 2016 00:00:00 GMT",790227,2118 Fredericksburg Rdj,San Antonio,TX,78201,Elmers,Elmers Board Mate Dual Tip Glue Pen,7.06,26000001403,26000001403


In [96]:
# Check the chage of the merged df
df.shape

(913000, 12)

In [97]:
# Set sale_id as the index and check the shape

df.set_index('sale_id', inplace=True)
df.shape

(913000, 11)

In [99]:
# Summarize the df
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 913000 entries, 1 to 913000
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   sale_amount    913000 non-null  float64
 1   sale_date      913000 non-null  object 
 2   store_address  913000 non-null  object 
 3   store_city     913000 non-null  object 
 4   store_state    913000 non-null  object 
 5   store_zipcode  913000 non-null  object 
 6   item_brand     913000 non-null  object 
 7   item_name      913000 non-null  object 
 8   item_price     913000 non-null  float64
 9   item_upc12     913000 non-null  object 
 10  item_upc14     913000 non-null  object 
dtypes: float64(2), object(9)
memory usage: 83.6+ MB


In [101]:
# # Save the data as the .csv
# df.to_csv('store_sales.csv')

#### `pd.merge(df1, df2, left_on='', right_on='').drop(columns='')`

In [38]:
# Load dfs from .csv

df_items = pd.read_csv('df_items.csv', index_col=0)
df_stores = pd.read_csv('df_stores.csv', index_col=0)
df_sales = pd.read_csv('df_sales.csv', index_col=0)

# Print the shape of the dfs
df_items.shape, df_stores.shape, df_sales.shape

((50, 6), (10, 5), (913000, 5))

In [40]:
df_items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [41]:
df_stores.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


In [42]:
df_sales.head()

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [45]:
df = pd.merge(df_sales, df_stores, left_on='store', right_on='store_id')
df = pd.merge(df, df_items, left_on='item', right_on='item_id')
df.drop(columns=['item', 'store'], inplace=True)

# Print the shape of merged df
df.shape

(913000, 14)

In [47]:
# Quick view of df
df.head()

,sale_amount,sale_date,sale_id,store_address,store_city,store_id,store_state,store_zipcode,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
2,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
3,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
4,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013


In [49]:
# # Write the df to .csv
# df.to_csv('store_data.csv')

### Build Help Function

In [50]:
def get_store_data():
    """
    Return store data either by reading from .csv or creating it
    """
    filename = 'store_data.csv'
    if os.path.isfile(filename):
        return pd.read_csv(filename, index_col=0)
    else: # loade df_items.csv files or create it
        if os.path.isfile('df_items.csv'):
            df_items = pd.read_csv('df_items.csv', index_col=0)
        else:
            df_items = get_pages('items')        
        # loade df_sales.csv files or create it
        if os.path.isfile('df_sales.csv'):
            df_sales = pd.read_csv('df_sales.csv', index_col=0)
        else:
            df_sales = get_pages('sales')        
        # loade df_stores.csv files or create it
        if os.path.isfile('df_stores.csv'):
            df_stores = pd.read_csv('df_stores.csv', index_col=0)
        else:
            df_stores = get_pages('stores')        
        # Merge all dfs
        df = pd.merge(df_sales, df_stores, left_on='store', right_on='store_id')
        df = pd.merge(df, df_items, left_on='item', right_on='item_id')
        df.drop(columns=['item', 'store'], inplace=True)
        df.to_csv('store_data.csv')
        return df

In [53]:
# Test function

store = get_store_data()
store.shape

(913000, 14)

In [54]:
# Quick view of the store data
store.head()

,sale_amount,sale_date,sale_id,store_address,store_city,store_id,store_state,store_zipcode,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
2,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
3,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
4,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013


6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [103]:
df_opsd = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')
df.shape

(4383, 5)

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4383 entries, 0 to 4382
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         4383 non-null   object 
 1   Consumption  4383 non-null   float64
 2   Wind         2920 non-null   float64
 3   Solar        2188 non-null   float64
 4   Wind+Solar   2187 non-null   float64
dtypes: float64(4), object(1)
memory usage: 171.3+ KB
